# _Influencia de la duración de piststops en carreras de F1_

* Hipótesis:  

> Mejor media de tiempos por carrera y temporada aumenta las posiciones en carrera y al final de temporada  

* hipótesis secundaria  

> También se sumple para los constructores  

* bonus: Diferencias entre conductores de una mismo equipo

<center><img src="F1_logo.jpg"  width="200"><img src="F170.jpg"  width="400"></center> <p>
<center><img src="pitStopAstonMartin.jpg"  width="600" height="300"></center>


### <span style="color:RED"> DATASET</span>

#### CSV's EXTRAIDOS DE ergast.com  

* Ofrece resultados de todas las carreras desde 1950 hasta hoy.  

* Paquete ergast_py  
https://pypi.org/project/ergast-py/

* API con datos actualizados tras cada carrera  
  
  * API documentación http://ergast.com/mrd/  
    
* Libreria propia para acceder a la API  
    pyERGAST


#### Ofrecen todo el paquete de csv hasta la carrera más reciente  
* f1db_csv.zip  
http://ergast.com/downloads/f1db_csv.zip  

* Diagrama de la BD  
http://ergast.com/images/ergast_db.png  

* Esquema de la BD  
http://ergast.com/docs/f1db_user_guide.txt  

Cada csv tiene una sola base de datos en UTF-8  
La primera línea son las cabeceras de las columnas. 




In [185]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sqlite3
import os
import re

* 13 csv

In [2]:
circuits=pd.read_csv(r'./circuits.csv')
constructor_results=pd.read_csv(r'./constructor_results.csv')
constructor_standings=pd.read_csv(r'./constructor_standings.csv')
driver_standings=pd.read_csv(r'./driver_standings.csv')
drivers=pd.read_csv (r'./drivers.csv')
lap_times=pd.read_csv(r'./lap_times.csv')
pit_stops=pd.read_csv(r'./pit_stops.csv')
qualifying=pd.read_csv(r'./qualifying.csv')
races=pd.read_csv(r'./races.csv')
results=pd.read_csv(r'./results.csv')
seasons=pd.read_csv(r'./seasons.csv')
sprint_results=pd.read_csv(r'./sprint_results.csv')
status=pd.read_csv(r'./status.csv')


* Para el primer proyecto EDA, los más interesantes son:  

    * pit stops                     | *pit_stops*  
    * resultados                    | *results.csv*  
    * conductores                   |  *drivers.csv*  
    * carreras                      |  *races.csv*  
    * resultados de constructores por campeonato   | *constructor_standings.csv*  
    * resultados de constructores por carrera   | *constructor_results.csv*  
    * status (finalizado/otros)     | *status.csv*
    * tiempo por vuelta             | *lap_times.csv*


---------------

## <center><span style="color:RED"> *PIT STOPS*</span></center>

---------------

In [3]:
pit_stops.head(2)

,raceId,driverId,stop,lap,time,duration,milliseconds
0,841,153,1,1,17:05:23,26.898,26898
1,841,30,1,1,17:05:52,25.021,25021


In [6]:
# cambiar tiempo en milisegundos a int
pit_stops["milliseconds"].astype(int).sample(2)

9027    22316
1075    21696
Name: milliseconds, dtype: int32

In [7]:
pit_stops.drop(['time', 'duration'], axis=1, inplace=True)
#quitar hora de parada y tiempo en segundos

In [8]:
pit_stops

,raceId,driverId,stop,lap,milliseconds
0,841,153,1,1,26898
1,841,30,1,1,25021
2,841,17,1,11,23426
3,841,4,1,12,23251
4,841,13,1,13,23842
...,...,...,...,...,...
9891,1105,847,2,45,22054
9892,1105,1,2,50,22731
9893,1105,815,2,50,21537
9894,1105,846,3,50,22062


In [85]:
# media de tiempo de paradas por carrera/conductor

pit_stops_mean=pit_stops.groupby(["raceId", "driverId"])["milliseconds"].mean()
pit_stops_mean

raceId  driverId
841     1           23213.0
        2           24046.0
        3           23716.0
        4           24055.0
        5           24865.0
                     ...   
1105    852         22092.5
        855         23159.5
        856         22249.0
        857         22616.0
        858         23205.0
Name: milliseconds, Length: 4837, dtype: float64

In [100]:
pit_stops.info()
# no tiene datos nulos
#datos cualitativos raceId, driverId, stop, lap
#datos cuantitativos: milliseconds

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9896 entries, 0 to 9895
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype
---  ------        --------------  -----
 0   raceId        9896 non-null   int64
 1   driverId      9896 non-null   int64
 2   stop          9896 non-null   int64
 3   lap           9896 non-null   int64
 4   milliseconds  9896 non-null   int64
dtypes: int64(5)
memory usage: 386.7 KB


---------------

## <center><span style="color:RED"> *RACES*</span></center>

---------------

In [ ]:
# quitar columnas de races

races.drop(['url', 'time', 'fp1_date', 'fp1_time', 'fp2_date', 'fp2_time', 'fp3_date', 'fp3_time', 'quali_date', 'quali_time', 'sprint_date', 'sprint_time'], axis=1, inplace=True)

In [105]:
races.sort_values("year").sample(2)

,raceId,year,round,circuitId,name,date
743,744,1961,7,14,Italian Grand Prix,1961-09-10
417,418,1986,15,32,Mexican Grand Prix,1986-10-12


In [82]:
races[["raceId", "year", "circuitId", "name"]][races.raceId>=842]

,raceId,year,circuitId,name
840,842,2011,2,Malaysian Grand Prix
841,843,2011,17,Chinese Grand Prix
842,844,2011,5,Turkish Grand Prix
843,845,2011,4,Spanish Grand Prix
844,846,2011,6,Monaco Grand Prix
...,...,...,...,...
1096,1116,2023,69,United States Grand Prix
1097,1117,2023,32,Mexico City Grand Prix
1098,1118,2023,18,São Paulo Grand Prix
1099,1119,2023,80,Las Vegas Grand Prix


In [168]:
races.sort_values("year").sample(2)

,raceId,year,round,circuitId,name,date
465,466,1983,15,30,South African Grand Prix,1983-10-15
738,739,1961,2,39,Dutch Grand Prix,1961-05-22


In [167]:
races.info()

#todo valores cualitativos, para unir con resultados y clasificar en gráficos

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1101 entries, 0 to 1100
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   raceId     1101 non-null   int64 
 1   year       1101 non-null   int64 
 2   round      1101 non-null   int64 
 3   circuitId  1101 non-null   int64 
 4   name       1101 non-null   object
 5   date       1101 non-null   object
dtypes: int64(4), object(2)
memory usage: 51.7+ KB


---------------

## <center><span style="color:RED"> *RESULTS*</span></center>

---------------

In [206]:
# resultados de cada carrera, identificador de carrera, conductor, constructor, posición, vueltas, tiempo total, puntos, vuelta rapida, tiempo de vuelta rapida
results.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2579 entries, 20822 to 25971
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   raceId           2579 non-null   int64  
 1   driverId         2579 non-null   int64  
 2   constructorId    2579 non-null   int64  
 3   grid             2579 non-null   int64  
 4   position         2579 non-null   object 
 5   points           2579 non-null   float64
 6   laps             2579 non-null   int64  
 7   milliseconds     2579 non-null   object 
 8   fastestLap       2579 non-null   object 
 9   fastestLapTime   2579 non-null   object 
 10  fastestLapSpeed  2579 non-null   object 
 11  statusId         2579 non-null   int64  
dtypes: float64(1), int64(6), object(5)
memory usage: 261.9+ KB


In [207]:
# De los pitstops cambió la norma para evitar repostajes por seguridad en 2009

# los pitstops.csv tiene información desde entonces en adelante.       carrera 842

# Nos quedamos con los resultados de conductores que han acabado las carreras.      statusId==1

results.sample(2)

,raceId,driverId,constructorId,grid,position,points,laps,milliseconds,fastestLap,fastestLapTime,fastestLapSpeed,statusId
25569,1082,840,117,17,10,1.0,70,5819979,64,1:17.421,202.782,1
24157,1008,1,131,1,1,25.0,71,5229066,70,1:11.795,216.065,1


In [208]:
results=results[(results.raceId>842)&(results.statusId==1)]
results.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2579 entries, 20822 to 25971
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   raceId           2579 non-null   int64  
 1   driverId         2579 non-null   int64  
 2   constructorId    2579 non-null   int64  
 3   grid             2579 non-null   int64  
 4   position         2579 non-null   object 
 5   points           2579 non-null   float64
 6   laps             2579 non-null   int64  
 7   milliseconds     2579 non-null   object 
 8   fastestLap       2579 non-null   object 
 9   fastestLapTime   2579 non-null   object 
 10  fastestLapSpeed  2579 non-null   object 
 11  statusId         2579 non-null   int64  
dtypes: float64(1), int64(6), object(5)
memory usage: 261.9+ KB


In [ ]:
# quitar columnas . Guardar info de vuelta rapida de cada conductor

results.drop(['positionText', 'positionOrder','number', 'resultId', 'time', 'rank'], axis=1, inplace=True)

In [212]:
results["milliseconds"].describe()

count        2579
unique       2577
top       5152531
freq            2
Name: milliseconds, dtype: object

In [211]:
# cambiar valores \N de milliseconds a 0
results["milliseconds"].replace('\\N', 0, inplace=True)

In [213]:
# cambiar tipo a entero
results['milliseconds'].astype(int)

20822    5818226
20823    5823424
20824    5825781
20825    5828226
20826    5831674
          ...   
25967    5347182
25968    5349818
25969    5351470
25970    5352359
25971    5353356
Name: milliseconds, Length: 2579, dtype: int32

In [214]:
# statusId=1 indica que el conductor terminó la carrera
status[["statusId","status"]][status.status=='Finished']

,statusId,status
0,1,Finished


In [215]:
# Resultados de cada carrera, identificadores de carrera, piloto, posición, puntos, vueltas, tiempos

results.sample(4)

,raceId,driverId,constructorId,grid,position,points,laps,milliseconds,fastestLap,fastestLapTime,fastestLapSpeed,statusId
24448,1022,842,5,13,9,2.0,44,5134878,21,1:49.700,229.848,1
23438,972,830,9,7,5,10.0,52,5349159,47,1:38.429,213.888,1
24701,1035,1,131,2,2,19.0,52,4793319,43,1:28.451,239.766,1
23485,974,839,10,15,12,0.0,78,6308065,52,1:16.482,157.072,1


In [ ]:
# merge con medias de tiempo de pitstops, races, y drivers

---------------

## <center><span style="color:RED"> *DRIVERS*</span></center>

---------------

In [ ]:
# unir nombre apellido
drivers['fullname']=drivers['forename']+" "+drivers['surname']

In [ ]:
# borrar columnas

drivers.drop(['nationality', 'code' 'dob', 'number', 'url', 'forename', 'surname'], axis=1, inplace=True)

In [224]:
# Información básica de cada conductor, id, alias, nombre completo
# todas cualitativas para mezclar con results

drivers[drivers.driverId==4]

,driverId,driverRef,fullname
3,4,alonso,Fernando Alonso


---------------

## <center><span style="color:RED"> *LAP TIMES*</span></center>

---------------

In [223]:
lap_times.info()

# sin valores nulos
# variable cuantitativa, tiempo en milisegundos por vuelta

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 546037 entries, 0 to 546036
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   raceId        546037 non-null  int64 
 1   driverId      546037 non-null  int64 
 2   lap           546037 non-null  int64 
 3   position      546037 non-null  int64 
 4   time          546037 non-null  object
 5   milliseconds  546037 non-null  int64 
dtypes: int64(5), object(1)
memory usage: 25.0+ MB


In [170]:
lap_times.sample(4)

,raceId,driverId,lap,position,time,milliseconds
95108,929,832,21,16,1:41.730,101730
156184,201,56,16,6,1:47.444,107444
189862,169,23,9,4,1:22.298,82298
431933,994,839,5,6,1:18.499,78499


In [220]:
# cambiar tipo de tiempo en milisegundos a int
lap_times["milliseconds"].astype(int)

0         98109
1         93006
2         92713
3         92803
4         92342
          ...  
546032    79035
546033    79096
546034    79502
546035    78922
546036    79271
Name: milliseconds, Length: 546037, dtype: int32

In [221]:
# media de tiempo por vuelta, por carrera y conductor
lap_times_race_mean=lap_times.groupby(['raceId', 'driverId'])['milliseconds'].mean()
lap_times_race_mean

raceId  driverId
1       1           97563.758621
        2           97635.672414
        3           97612.172414
        4           97597.637931
        6           91822.235294
                        ...     
1105    852         81035.696970
        855         81057.848485
        856         81281.030769
        857         81264.846154
        858         81994.107692
Name: milliseconds, Length: 10285, dtype: float64

In [172]:
lap_times.sample()

,raceId,driverId,lap,position,time,milliseconds
350128,8,9,17,14,1:23.718,83718


In [219]:
plt.bar(lap_times_race_mean[lap_times_race_mean.raceId==1000], lap_times_race_mean)

AttributeError: 'Series' object has no attribute 'raceId'